# 1.Modelo yolo, treinamentos e testes de resultado



In [ ]:
from google.colab import drive   # integração drive e colab
drive.mount('/content/drive')

In [ ]:
! git clone https://github.com/ultralytics/yolov5.git

In [ ]:
! pip install -r yolov5/requirements.txt   # instalando requirements para rodar yolo

In [ ]:
yaml_content = """
train: /content/drive/MyDrive/FIAP/FIAP_NEXT/2025/images/train/
val: /content/drive/MyDrive/FIAP/FIAP_NEXT/2025/images/val/

nc: 2 # number of classes
names: # Define here the labels of your dataset
  0: "Mouse"
  1: "Controle"
"""

with open("MouseControle1.yaml", "w") as f:
    f.write(yaml_content)

In [ ]:
!python yolov5/train.py --data MouseControle1.yaml --weights yolov5s.pt --img 640 --epochs 60   # testes com 40 e depois 60 epochs

In [ ]:
# rodando testes de acurácia

import os
import subprocess

def get_latest_train_run_folder():
    subfolders = [f.path for f in os.scandir('yolov5/runs/train') if f.is_dir()]
    latest_folder = max(subfolders, key=os.path.getctime, default=None)
    return latest_folder

latest_run = get_latest_train_run_folder()
result = subprocess.run(f'python yolov5/detect.py --weights {latest_run}/weights/best.pt --img 640 --source drive/MyDrive/FIAP/FIAP_NEXT/2025/images/test/ --data yolov5/data/MouseControle1.yaml', shell=True, capture_output=True, text=True)
if latest_run:
    # COMANDO
    result = subprocess.run(f'python yolov5/detect.py --weights {latest_run}/weights/best.pt --img 640 --source drive/MyDrive/FIAP/FIAP_NEXT/2025/images/test/ --data yolov5/data/MouseControle1.yaml', shell=True, capture_output=True, text=True)
    print(result.stdout)
    print(result.stderr)
else:
    print("Não foi possível encontrar a pasta de treinamento mais recente.")

# 2.CNN para classificar classe de imagens

In [ ]:
!pip install -q ultralytics # ultralytics fornece APIs de YOLO modernas (v8) e permite treino/avaliação fáceis
!pip install -q yolov5 # utilitário que pode ajudar se preferir YOLOv5
!pip install -q tensorflow
!pip install -q opencv-python-headless
!pip install -q seaborn matplotlib scikit-learn pandas
!pip install numpy

In [ ]:
import os
from pathlib import Path
DATASET_PATH = Path('/content/dataset') # ajuste se necessário
IMG_DIR = DATASET_PATH / 'images'
LABELS_DIR = DATASET_PATH / 'labels'

In [ ]:
for p in [IMG_DIR / 'train', IMG_DIR / 'val', LABELS_DIR / 'train', LABELS_DIR / 'val']:
    p.parent.mkdir(parents=True, exist_ok=True)


print('Dataset root:', DATASET_PATH)
print('Train images example dir:', IMG_DIR / 'train')

In [ ]:
YOLO_ADAPTAVEL_WEIGHTS = '/content/entrega1_weights.pt' # substitua pelo seu arquivo .pt se existir
from ultralytics import YOLO
if os.path.exists(YOLO_ADAPTAVEL_WEIGHTS):
    model_adapt = YOLO(YOLO_ADAPTAVEL_WEIGHTS)
    print('Modelo adaptável carregado:', YOLO_ADAPTAVEL_WEIGHTS)
else:
    model_adapt = None
    print('Nenhum peso adaptável encontrado; deixe YOLO_ADAPTAVEL_WEIGHTS apontar para seu .pt')

In [ ]:
if model_adapt is not None:
    example_img = next((IMG_DIR / 'val').glob('*.jpg'), None)
    if example_img:
        print('Executando inferência em:', example_img)
        res = model_adapt.predict(str(example_img))
        display(res[0].plot())
    else:
        print('Nenhuma imagem de validação encontrada para inferência de exemplo.')
else:
    print('Pule esta célula — modelo adaptável não está disponível.')

In [ ]:
yolo_data_yaml = {
'path': str(DATASET_PATH),
'train': 'images/train',
'val': 'images/val',
'nc': 1, # número de classes; altere se tiver mais classes
'names': ['classe0'] # altere para os nomes reais das classes
}


import yaml
with open('data.yaml', 'w') as f:
    yaml.dump(yolo_data_yaml, f)


print('Arquivo data.yaml criado. Confira nc e names antes de treinar!')

In [ ]:
# Aqui usamos a API Ultralytics para treinar um modelo 'yolov8n' (pequeno, rápido). Se quiser usar yolov5, comente e use a seção alternativa.

EPOCHS = 30
BATCH = 8
IMG_SIZE = 640
MODEL_NAME = 'yolov8n.pt' # modelo pequeno pré-treinado (Ultralytics)

In [ ]:
# Execute o treino (pode demorar — adapte EPOCHS/BATCH)
from ultralytics import YOLO
model_baseline = YOLO(MODEL_NAME)


In [ ]:
# Treine; os resultados ficam em ./runs/train

print('Iniciando treino YOLO baseline — ajuste EPOCHS/BATCH/IMG_SIZE conforme o seu ambiente')
model_baseline.train(data='data.yaml', epochs=EPOCHS, imgsz=IMG_SIZE, batch=BATCH, name='baseline_yolo')

In [ ]:
# Avaliar o modelo treinado (usa val set definido em data.yaml)

results = model_baseline.val() # retorna dicionário com métricas, incluindo mAP
print('Métricas do YOLO baseline:')
print(results)

In [ ]:
# Salve pesos finais para comparação

baseline_weights = 'runs/train/baseline_yolo/weights/best.pt'
print('Pesos salvo em:', baseline_weights)

In [ ]:
import cv2


IMG_SIZE_CLS = (128,128)
BATCH_CLS = 32


train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=15, width_shift_range=0.1,
height_shift_range=0.1, horizontal_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(str(CROPS_DIR/'train'), target_size=IMG_SIZE_CLS, batch_size=BATCH_CLS, class_mode='categorical')
val_generator = val_datagen.flow_from_directory(str(CROPS_DIR/'val'), target_size=IMG_SIZE_CLS, batch_size=BATCH_CLS, class_mode='categorical')


NUM_CLASSES = train_generator.num_classes
print('Número de classes (classification):', NUM_CLASSES)

In [ ]:
# Definir e treinar uma CNN simples do zero

from tensorflow.keras import layers, models, optimizers, callbacks


def build_simple_cnn(input_shape=(128,128,3), num_classes=NUM_CLASSES):
model = models.Sequential([
layers.Conv2D(32,3,activation='relu',input_shape=input_shape),
layers.MaxPooling2D(2),
layers.Conv2D(64,3,activation='relu'),
layers.MaxPooling2D(2),
layers.Conv2D(128,3,activation='relu'),
layers.MaxPooling2D(2),
layers.Flatten(),
layers.Dense(128,activation='relu'),
layers.Dropout(0.5),
layers.Dense(num_classes,activation='softmax')
])
model.compile(optimizer=optimizers.Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
return model


cnn_model = build_simple_cnn()
cnn_model.summary()



EPOCHS_CLS = 25
callbacks_list = [callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
history = cnn_model.fit(train_generator, validation_data=val_generator, epochs=EPOCHS_CLS, callbacks=callbacks_list)



cnn_model.save('cnn_from_scratch.h5')
print('Modelo de classificação salvo como cnn_from_scratch.h5')

In [ ]:
# Avaliação da CNN e matrizes

import numpy as np
from sklearn.metrics import classification_report, confusion_matrix


Prever no val set
val_generator.reset()
preds = cnn_model.predict(val_generator)
y_pred = np.argmax(preds, axis=1)
y_true = val_generator.classes


print('Classification report:')
print(classification_report(y_true, y_pred, target_names=list(val_generator.class_indices.keys())))


cm = confusion_matrix(y_true, y_pred)
print('Matriz de confusão:\n', cm)

In [ ]:
import time



def measuretime(fn, inputs, repeat=20):
times = []
for  in range(repeat):
start = time.time()
fn(inputs)
times.append(time.time()-start)
return float(np.mean(times)), float(np.std(times))

In [ ]:
# Medir inferência YOLO baseline

if 'model_baseline' in globals() and model_baseline is not None:
sample_imgs = list((IMG_DIR/'val').glob('.jpg'))[:10]
def runyolo(imgs):
for p in imgs:
 = model_baseline.predict(str(p), verbose=False)
mean_yolo, std_yolo = measure_time(run_yolo, sample_imgs, repeat=3)
print(f'YOLO baseline — tempo médio (para {len(sample_imgs)} imgs): {mean_yolo:.3f}s (std {std_yolo:.3f})')
else:
print('Modelo YOLO baseline não disponível para medir tempo.')


In [ ]:
# Medir inferência CNN (classificação)

sample_img = next((CROPS_DIR/'val').glob('**/.jpg'), None)
if sample_img is not None:
import PIL.Image as Image
img = Image.open(sample_img).resize(IMG_SIZE_CLS)
arr = np.array(img)/255.0
def runcnn():
_ = cnn_model.predict(np.expand_dims(arr,0))
mean_cnn, std_cnn = measure_time(run_cnn, None, repeat=20)
print(f'CNN classificação — tempo médio por imagem: {mean_cnn:.4f}s (std {std_cnn:.4f})')
else:
print('Não há crops para medir inferência da CNN.')